<a href="https://colab.research.google.com/github/Traivok/tcc/blob/develop/scripts/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi
! pip install ultralytics --upgrade
! pip install wandb
! pip install roboflow
! pip install scikit-learn
! pip install python-dotenv

Sun Nov 17 16:29:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import ultralytics
import warnings
import torch
import os

In [ ]:
%matplotlib inline
ultralytics.checks()

In [8]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

from dotenv import load_dotenv
load_dotenv()

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

if IN_COLAB:
  from google.colab import userdata
  ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')



In [9]:
# Check if running on a MacBook with M1 chip
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) for hardware acceleration on MacBook M1.")
# Check if CUDA is available
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA for hardware acceleration.")
else:
    device = torch.device("cpu")
    warnings.warn("Using CPU for computation. This may be slower than using MPS or CUDA.", UserWarning)

Using CUDA for hardware acceleration.


In [10]:
base_path = os.getcwd()
print(f"Current working directory: {base_path}")

# Example usage with Ultralytics
model_list =  [
    #  'yolov3u.pt',
    #  'yolov5mu.pt',
     'yolov8m.pt',
    #  'yolov9m.pt',
    #  'yolov10m.pt',
    #  'yolo11m.pt',
]

for model_name in model_list:
        print(model_name)
        model = ultralytics.YOLO(model_name).to(device)

Current working directory: /content
yolov8m.pt


In [2]:
from roboflow import Roboflow

print(f"Roboflow API key: {ROBOFLOW_API_KEY}")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("ze").project("tcc-bbcjn")
version = project.version(1)
dataset = version.download("yolov8")

NameError: name 'ROBOFLOW_API_KEY' is not defined

In [ ]:
! wandb login WANDB_API_KEY --verify

In [12]:
from google.colab import drive
import shutil

# Monta o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback
from ultralytics import YOLO

DATASET = dataset.location + "/data.yaml"
MODEL_FILENAME=model_list[0] ## should be yolov8
PATIENCE=30
EPOCHS=1
IMGSZ=(640, 640)

MODEL_NAME = os.path.splitext(MODEL_FILENAME)[0]
RUN_NAME=f'tcc_{MODEL_NAME}'

model = YOLO(MODEL_FILENAME)

add_wandb_callback(model, enable_model_checkpointing=True)
wandb.init(project=RUN_NAME)

results = model.train(
    task='detect',
    data=DATASET,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=32,
    patience=10,
    project='tcc',
    name=RUN_NAME,
    save=True,
    save_period=5,
    cache=True,
    plots=True,
    seed=0)

wandb.finish()

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


NameError: name 'dataset' is not defined

In [3]:
# Define o caminho de destino no Google Drive
destination_dir = f'/content/drive/My Drive/tcc/'
# destination_path = os.path.join(destination_dir, 'best.pt')


# Verifica se o diretório existe; se não, cria-o
import os
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copia o arquivo para o destino no Google Drive
from distutils.dir_util import copy_tree

copy_tree(f'/content/tcc', destination_dir)

print("Arquivo enviado para o Google Drive com sucesso!")

Arquivo enviado para o Google Drive com sucesso!
